In [1]:
import xarray as xr
import os
from glob import glob

# =====================================================
# PATHS
# =====================================================
CLIMATE_DIR = "/content/drive/MyDrive/Research Works - Development/Work 2 - SPI SPEI/Data/raster/combined_climate_growing_season/Climate_variables"
DROUGHT_DIR = "/content/drive/MyDrive/Research Works - Development/Work 2 - SPI SPEI/Data/raster/combined_climate_growing_season/Drought_variables"
OUTPUT_DIR  = "/content/drive/MyDrive/Research Works - Development/Work 2 - SPI SPEI/Data/raster/Data for Dev/seasonal_aggregated"

os.makedirs(OUTPUT_DIR, exist_ok=True)

YEARS = range(1991, 2024)
SEASON_MONTHS = [5, 6, 7, 8, 9, 10]   # May–Oct


In [2]:
# Climate aggregation rules
CLIMATE_SUM_VARS = ["monthly_rain"]
CLIMATE_MEAN_VARS = [
    "max_temp", "min_temp", "radiation",
    "vp", "vp_deficit", "rh_tmax", "rh_tmin"
]

# Drought variables (all aggregated as mean or min)
DROUGHT_VARS = [
    "spi_1", "spi_3", "spi_6", "spi_12",
    "spei_1", "spei_3", "spei_6", "spei_12"
]


In [5]:
for year in YEARS:
    print(f"🔄 Processing {year}")

    # -------------------------------------------------
    # Load datasets
    # -------------------------------------------------
    climate_ds = xr.open_dataset(
        os.path.join(CLIMATE_DIR, f"silo_growing_season_{year}.nc")
    )

    drought_ds = xr.open_dataset(
        os.path.join(DROUGHT_DIR, f"drought_growing_season_{year}.nc")
    )

    # -------------------------------------------------
    # Select May–Oct only
    # -------------------------------------------------
    climate_ds = climate_ds.sel(
        time=climate_ds.time.dt.month.isin(SEASON_MONTHS)
    )

    drought_ds = drought_ds.sel(
        time=drought_ds.time.dt.month.isin(SEASON_MONTHS)
    )

    seasonal_vars = {}

    # -------------------------------------------------
    # Climate aggregation
    # -------------------------------------------------
    for var in CLIMATE_SUM_VARS:
        seasonal_vars[f"{var}_MJJAO"] = climate_ds[var].sum(dim="time")

    for var in CLIMATE_MEAN_VARS:
        seasonal_vars[f"{var}_MJJAO"] = climate_ds[var].mean(dim="time")

    # -------------------------------------------------
    # Drought aggregation (MEAN over season)
    # -------------------------------------------------
    for var in DROUGHT_VARS:
        seasonal_vars[f"{var}_MJJAO"] = drought_ds[var].mean(dim="time")

    # -------------------------------------------------
    # Combine into one Dataset
    # -------------------------------------------------
    seasonal_ds = xr.Dataset(seasonal_vars)

    # -------------------------------------------------
    # Metadata
    # -------------------------------------------------
    seasonal_ds.attrs.update({
        "title": "Seasonal Aggregated Climate & Drought Indices (May–Oct)",
        "year": year,
        "aggregation": "Rain=sum, others=mean, SPI/SPEI=mean",
        "season": "May–October",
        "created_by": "xarray seasonal aggregation"
    })

    # -------------------------------------------------
    # Save
    # -------------------------------------------------
    out_file = os.path.join(OUTPUT_DIR, f"seasonal_features_{year}.nc")
    seasonal_ds.to_netcdf(out_file)

    print(f"✅ Saved {out_file}")

print("🎉 Seasonal aggregation complete!")


🔄 Processing 1991
✅ Saved /content/drive/MyDrive/Research Works - Development/Work 2 - SPI SPEI/Data/raster/Data for Dev/seasonal_aggregated/seasonal_features_1991.nc
🔄 Processing 1992
✅ Saved /content/drive/MyDrive/Research Works - Development/Work 2 - SPI SPEI/Data/raster/Data for Dev/seasonal_aggregated/seasonal_features_1992.nc
🔄 Processing 1993
✅ Saved /content/drive/MyDrive/Research Works - Development/Work 2 - SPI SPEI/Data/raster/Data for Dev/seasonal_aggregated/seasonal_features_1993.nc
🔄 Processing 1994
✅ Saved /content/drive/MyDrive/Research Works - Development/Work 2 - SPI SPEI/Data/raster/Data for Dev/seasonal_aggregated/seasonal_features_1994.nc
🔄 Processing 1995
✅ Saved /content/drive/MyDrive/Research Works - Development/Work 2 - SPI SPEI/Data/raster/Data for Dev/seasonal_aggregated/seasonal_features_1995.nc
🔄 Processing 1996
✅ Saved /content/drive/MyDrive/Research Works - Development/Work 2 - SPI SPEI/Data/raster/Data for Dev/seasonal_aggregated/seasonal_features_1996.n